# 📘 Radioactive Decay: Half-Life Determination> Fit exponential decay to nuclear decay data⏱️ **15-20 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Physics** | 🔬 **Nuclear**---

## 🔬 Domain Background**Model:** $N(t) = N_0 e^{-\lambda t}$**Where:**- $N_0$: Initial number of nuclei- $\lambda$: Decay constant- $t$: Time**Applications:** Dating, nuclear medicine, radiation safety---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit


def radioactive_decay(t, N0, lambda_decay):
    return N0 * jnp.exp(-lambda_decay * t)

## Generate Decay Data

In [ ]:
N0_true = 10000
lambda_true = 0.1  # 1/s
t = np.linspace(0, 50, 30)
N_true = radioactive_decay(t, N0_true, lambda_true)
N = N_true + np.random.normal(0, 100, len(t))

plt.semilogy(t, N, 'o', alpha=0.5, label='Measured counts')
plt.semilogy(t, N_true, 'r--', label='True decay')
plt.xlabel('Time (s)')
plt.ylabel('Count Rate')
plt.legend()
plt.tight_layout()
plt.tight_layout()
display(fig)
plt.close(fig)


## Determine Half-Life

In [ ]:
popt, pcov = curve_fit(radioactive_decay, t, N, p0=[9500, 0.09])
N0_fit, lambda_fit = popt
perr = np.sqrt(np.diag(pcov))

t_half = np.log(2) / lambda_fit
t_half_err = t_half * (perr[1] / lambda_fit)

print(f'Decay constant (λ): {lambda_fit:.4f} ± {perr[1]:.4f} s⁻¹')
print(f'Half-life (t₁/₂):    {t_half:.2f} ± {t_half_err:.2f} s')
print(f'\nAfter 1 half-life: {50}% remain')
print(f'After 2 half-lives: {25}% remain')

## Key Insights1. **Linear on log scale** confirms exponential decay2. **Half-life constant** - characteristic of isotope3. **Statistical fluctuations** inherent in nuclear decay---